In [11]:
from ultralytics import YOLO
from os import path, listdir, makedirs
from shutil import rmtree, copy
import numpy as np
import time
import random
import cv2

### Prepare directories

In [9]:
# Load all components
COMPONENTS = listdir("./datasets/full")
COMPONENTS.remove("current")
print(f"Components: {COMPONENTS}")

if path.exists("./datasets/partial"):
    rmtree("./datasets/partial")
makedirs("./datasets/partial")
for component in COMPONENTS:
    makedirs(f"./datasets/partial/{component}")
    makedirs(f"./datasets/partial/{component}/imgs")
    makedirs(f"./datasets/partial/{component}/labels")

Components: ['capacitor', 'ceramic_capacitor', 'film_capacitor', 'inductor', 'led', 'resistor', 'template', 'wire']


### Load classifer model

In [6]:
runs = listdir("./logs")
runs.sort()
latest_run = runs[-1]
modelPath = f"./logs/{latest_run}/weights/best.pt"
print(f"Latest run: {latest_run}, loading model from {modelPath}")
bestModel = YOLO(modelPath).to('cuda')
model = bestModel

Latest run: all_run-20240521-124737, loading model from ./logs/all_run-20240521-124737/weights/best.pt
Components: ['capacitor', 'ceramic_capacitor', 'film_capacitor', 'inductor', 'led', 'resistor', 'template', 'wire']


### Generate partial dataset

In [12]:
def crop_image_to_bbox(image, bbox):
    """
    Crop the image to the region within the bounding box.

    Args:
    - image (np.array): The input image.
    - bbox (tensor): The coordinates of the bounding box in xyxyxyxy format.

    Returns:
    - cropped_image (np.array): The cropped image.
    """
    # Convert tensor to numpy array and ensure it's on the CPU
    bbox = bbox.cpu().numpy()[0]

    # Compute the width and height of the bounding box
    width = int(np.linalg.norm(bbox[0] - bbox[1]))
    height = int(np.linalg.norm(bbox[1] - bbox[2]))

    # Compute the center of the bounding box
    center = np.mean(bbox, axis=0).astype(int)

    # Compute the rotation angle of the bounding box
    angle = np.degrees(np.arctan2(bbox[1, 1] - bbox[0, 1], bbox[1, 0] - bbox[0, 0]))
    rotation_matrix = cv2.getRotationMatrix2D((int(center[0]), int(center[1])), angle, 1.0)

    # Apply the rotation to the image
    rotated_image = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))

    # Get the bounding box in the rotated image
    x, y = center - [width // 2, height // 2]
    cropped_image = rotated_image[y:y+height, x:x+width]

    return cropped_image

# Inference on all components
for component in COMPONENTS:
    imgs = listdir(f"./datasets/full/{component}/imgs")
    print(f"There are {len(imgs)} images in {component}")
    for image in imgs:
        result = model.predict(f"./datasets/full/{component}/imgs/{image}")
        cropped_img = crop_image_to_bbox(result[0].orig_img, result[0].obb.xyxyxyxy)
        cv2.imwrite(f"./datasets/partial/{component}/imgs/{image}", cropped_img)

There are 164 images in capacitor

image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\full\capacitor\imgs\obb_capacitor_100uF-16V_0.png: 480x640 158.5ms
Speed: 3.5ms preprocess, 158.5ms inference, 842.0ms postprocess per image at shape (1, 3, 480, 640)
[196 187]

image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\full\capacitor\imgs\obb_capacitor_100uF-16V_1.png: 480x640 21.0ms
Speed: 3.0ms preprocess, 21.0ms inference, 5.5ms postprocess per image at shape (1, 3, 480, 640)
[343 392]

image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\full\capacitor\imgs\obb_capacitor_100uF-16V_10.png: 480x640 21.0ms
Speed: 2.5ms preprocess, 21.0ms inference, 5.0ms postprocess per image at shape (1, 3, 480, 640)
[253 179]

image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\full\capacitor\img